# Importing packages and load CNN

In [ ]:
# Load the extension
%load_ext autoreload
# Autoreload all modules
%autoreload 2

In [ ]:
from funcs import *

# Telling the notebook to make plots inline.
%matplotlib inline
plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=12)

## Start camera and DAC

In [ ]:
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
print("Using Camera ", camera.GetDeviceInfo().GetModelName())
camera.Open()

# Sanity checks
print('Camera Auto Gain is :', camera.GainAuto())
if camera.GainAuto() == 'Off':
    print('Safe to go ahead with the code..')
else:
    print('***Warning: Camera Auto Gain is ON!***\nTurn it off using Pylon software.')
camera.ExposureTimeAbs = Exposure

# Start DAC
start_channel = 1
bus = busworks.BusWorks_DAC()
bus.start()
print("DAC on IP: ", bus.address)

## Initial State

In [ ]:
Img_initial = Capture_image(1000)
print('Initial condition: {}'.format(Img_initial.sum()/n_pixl**2))
plt.imshow(Img_initial[::-1], cmap=cm.binary_r)
plt.colorbar()

In [ ]:
# Initial condition of beam
current_beam_status = np.zeros(num_params)

## Alignment loop

In [ ]:
# Creating the initial population of deltas
new_pop_deltas = sample_d(Range, shape=pop_size)
print("\n\nRange:", Range, "x waist")
sess = tf.Session()

gen = 0
dont_alter_z_cm = False
t0 = time.time()

# GA loop
for gen in range(1):
    # Shrink range
    Range *= shrink_factor
    print("\n\n Gen: {} Range: {} x waist".format(gen, Range))
    if dont_alter_z_cm:
        new_pop_deltas[:,-1] = 0.
    if gen == 0:
        current_beam_status, new_pop_deltas, fitness = calc_pop_fitness(current_beam_status, \
        new_pop_deltas, fitness, only_offsprings=False)
    else:
        current_beam_status, new_pop_deltas, fitness = calc_pop_fitness(current_beam_status, \
        new_pop_deltas, fitness, only_offsprings=True)
    # Selecting the best parents in the population for mating.
    current_beam_status, parents, parents_fitness, Img_is_saturated, image = \
        select_mating_pool(current_beam_status, new_pop_deltas, fitness, num_parents_mating, t0, gen, \
                           show_the_best=True, save_best=False)
    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, (pop_per_gen - num_parents_mating, num_params))
    # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover, Range)
    # Creating the new population based on the parents and offspring.
    new_pop_deltas[:num_parents_mating, :] = parents
    new_pop_deltas[num_parents_mating:, :] = offspring_mutation
    fitness[:num_parents_mating] = parents_fitness
    fitness[num_parents_mating:] *= 0.
    # Scan the cavity and regain the max power point in z_CM
    current_beam_status, new_pop_deltas, _ = scan_cavity(current_beam_status, new_pop_deltas, \
                                                           Range, size, show_fig=True)
    # Check mode at every 5th gen
    if gen%5 == 0 and gen > 0:
        mode = Find_mode2(img_loc, separation1=5, corner=0, show_fig=True, show_basis=True)
        print('Mode is {}'.format(mode))
        if np.all(mode == (0,0)):
            print('Fundamental mode identified! Exiting..')
            break
        else:
            # taking a jump in z_CM to make the cavity resonant to fundamental mode only
            current_beam_status, new_pop_deltas, _ = jump_2_fundamental(current_beam_status, new_pop_deltas, \
                                                                        mode, show_fig=True)
            # freezing z_CM 
            dont_alter_z_cm = True

#     if (gen>7) and (Img_is_saturated):
#         image = pad_and_resize_image(image)
#         # time to check the mode
#         mode = read_mode(image)
#         print('Mode is {}'.format(mode))
#         if (int(mode[0][3]) + int(mode[0][5])) == 0:
#             print('Fundamental mode identified! Exiting..')
#             break

In [ ]:
Img1 = Capture_image(700)
print(Img1.sum()/n_pixl**2)
plt.imshow(Img1[::-1], cmap=cm.binary_r)
plt.colorbar()

In [ ]:
# Stop camera and DAC
camera.Close()
bus.stop()

## Make gif of the images

In [ ]:
import imageio

files = os.listdir(ImagesFolder)
files.sort()
images = []
for filename in files:
    images.append(imageio.imread(ImagesFolder+'/'+filename))
imageio.mimsave(ImagesFolder + '/movie.gif', images, duration=0.5)